This notebook will be used for the Coursera captsone project

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Part 1: Segmenting and Clustering Neighborhoods in Toronto

Scraping the wikipedia page and cleaning the data to obtain the proper dataframe.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
df.shape

(103, 3)

# Part 2: Segmenting and Clustering Neighborhoods in Toronto

Using CSV method as geocoder would not work

In [5]:
import csv
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'

Convert to dataframe and fix postal code heading

In [6]:
df_coord = pd.read_csv(url,header=0)
df_coord.rename({'Postal Code':'PostalCode'}, axis=1,inplace=True)
df_coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge dataframes

In [7]:
df = df.merge(df_coord)

df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Part 3: Segmenting and Clustering Neighborhoods in Toronto

In [8]:
from sklearn.cluster import KMeans

In [9]:
!pip3 install folium

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 8.3 MB/s  eta 0:00:01


In [10]:
import folium

Cell with Foursquare API info is hidden

In [15]:
# The code was removed by Watson Studio for sharing.

Create Unclustered Map of Downtown Toronto Post Codes

In [76]:
df_toronto = df.loc[df['Borough']=='Downtown Toronto'].reset_index()
latitude = df_toronto.iloc[0]['Latitude']
longitude = df_toronto.iloc[0]['Longitude']

In [77]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['PostalCode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Obtain Foursquare Location data for downtown Toronto postal codes and coordinates

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
toronto_venues = getNearbyVenues(names=df_toronto['PostalCode'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

M5A
M5B
M5C
M5E
M5G
M6G
M5H
M5J
M5K
M5L
M5S
M5T
M5V
M4W
M4X
M5X
M4Y


In [52]:
toronto_venues.shape

(1120, 7)

In [53]:
toronto_venues.head()

,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [56]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['Postal Code']

# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by postal code and mean frequency of each venue type

In [57]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4W,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.25,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4X,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,M4Y,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.00,...,0.012658,0.012658,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.025316
3,M5A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.00,...,0.044444,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.022222
4,M5B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.020000,0.000000,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.000000,0.000000
5,M5C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022989,0.000000,0.00,...,0.011494,0.000000,0.00,0.00,0.00,0.011494,0.000000,0.000000,0.011494,0.000000
6,M5E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.016949,0.000000,0.000000,0.000000,0.000000
7,M5G,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.00,0.00,0.014706,0.000000,0.000000,0.000000,0.014706
8,M5H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,...,0.010000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.000000,0.000000
9,M5J,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000


Run K means clustering on the dataset

In [67]:
toronto_grouped_clustering = toronto_grouped.drop('PostalCode',1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_

array([0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 4, 2, 3], dtype=int32)

In [81]:
toronto_clustered = df_toronto.join(pd.DataFrame(kmeans.labels_,columns=['Cluster']))
toronto_clustered

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
1,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2
2,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2
3,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2
4,24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2
5,25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2
6,30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2
7,36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,2
8,42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,2
9,48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,2


Visualize clustered Postal Codes

In [83]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [84]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clustered['Latitude'], toronto_clustered['Longitude'], toronto_clustered['Neighborhood'], toronto_clustered['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters